In [1]:
"""
Authors: Ritvik Kapila, Gauri Gupta
"""

import xlrd 
import pandas as pd

In [2]:
def xlsx_to_csv(input_filename, list_of_number_of_group_by_columns, list_of_sheets, output_filename):
    wb = xlrd.open_workbook(input_filename) 
    
    l = list_of_number_of_group_by_columns
    
#     If the list_of_sheets is entered as empty list, we include all the sheets in the csv
    
    if(len(list_of_sheets) == 0):
        list_of_sheets = list(range(len(wb.sheet_names())))
    
#     Forming a dictionary 'col' having keys as column names and values as the respective unique number column_counter
#     This helps in appending rows by changing values in an empty list of length n = len(col.keys()) at only certain indices.

    col = {}
    col['Class'] = 0
    column_counter = 1
    
    for i in range(len(wb.sheet_names())):
        if(list_of_sheets.count(i) > 0):
            sheet = wb.sheet_by_index(i)

            for j in range(l[i]):
                try:
                    if(col[sheet.cell_value(1,j)]):
                        continue
                except KeyError:
                    col[sheet.cell_value(1,j)] = column_counter
                    column_counter += 1

    col['Parameter'] = column_counter
    column_counter += 1
    col['Value'] = column_counter
    n = len(col.keys())

#     Initializing an empty list of length n and changing the values corresponding to that particular sheet
#     Changing values of the group by columns and adding parameter and value for all non-group by columns
    
    total_rows = []
    for i in range(len(wb.sheet_names())):
        if(list_of_sheets.count(i) > 0):    
            sheet = wb.sheet_by_index(i)

            for j in range(2, sheet.nrows):

                group_by_col_list = {}
                for k in range(l[i]):
                    group_by_col_list[sheet.cell_value(1, k)] = sheet.cell_value(j, k)

                group_by_row = [''] * n
                for k in group_by_col_list:
                    group_by_row[col[k]] = group_by_col_list[k]

                for k in range(l[i], sheet.ncols):
                    row = group_by_row.copy()
                    row[col['Class']] = wb.sheet_names()[i]
                    row[col['Parameter']] = sheet.cell_value(1, k)
                    row[col['Value']] = sheet.cell_value(j, k)
                    total_rows += [row]

#     Forming a column list from the dictionary keys

    columns = [''] * n
    for key in col:
        columns[col[key]] = key
        
#     Output file in the form of a csv

    df = pd.DataFrame(total_rows, columns = columns)
    df.to_csv(output_filename, index = False)



In [8]:
# list_of_number_of_group_by_columns is the number of initial columns which are taken as group by columns.
# configurable_list_of_sheets is the list of indices of the sheets that we want to include in our csv file. 
# configurable_list_of_sheets can be left empty in order to include all the sheets.

list_of_number_of_group_by_columns = [5, 6, 3, 3, 5, 5, 3]
configurable_list_of_sheets = [0, 2]

xlsx_to_csv('xml_dump_in_XL.xlsx', list_of_number_of_group_by_columns, configurable_list_of_sheets, 'Result_Sheet_[0,2].csv')